In [5]:
### The primary script taken from Vasco Müller ###
### The script is adapted to work with EERIE IFS-FESOM data by Rohit Ghosh ###

import pyfesom2 as pf
import xarray as xr
import numpy as np

In [6]:
mesh_path='/work/ab0995/a270088/meshes/NG5/'
diag = xr.open_dataset((mesh_path+'fesom.mesh.diag.nc'))  # fesom.mesh.diag.nc contains the dx/dy operators (basically just vectors of the same size as u/v)
ddx = diag['gradient_sca_x']
ddy = diag['gradient_sca_y']
elem = (diag['elements']-1).T  #element indices are saved in Fortran format, starting from 1 instead of 0

In [ ]:
%%time

depth=100 # 100m 
iz = np.argmin(np.abs(diag.nz1.values+depth)) #find the closest level in the model, depth in diag.nz1 is negative, that is why I use +depth

# Define the years and months
years = ['1971']  # Example list of years
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']  # List of months

# Loop over years and months
for year in years:
    for mon in months:
        data_path='/work/bm1344/a270228/EERIE_NextG_Hackathon/IFS-FESOM_CONTROL-1950/tco1279-NG5/{year}/FESOM/native/daily/'

        # Use string formatting to replace {year} in the data_path
        data_path = data_path.format(year=year)

        unod = xr.open_mfdataset((data_path+f'fesom_unod_{year}{mon}*-{year}{mon}*_daily-NG5.nc'),chunks={'time':40,'nod2':200})['unod'].astype('float32').isel(nz1=iz)
        vnod = xr.open_mfdataset((data_path+f'fesom_vnod_{year}{mon}*-{year}{mon}*_daily-NG5.nc'),chunks={'time':40,'nod2':200})['vnod'].astype('float32').isel(nz1=iz)
        time= xr.open_mfdataset((data_path+f'fesom_unod_{year}{mon}*-{year}{mon}*_daily-NG5.nc'),chunks={'time':40,'nod2':200})['time'].astype('float32')

        n= time.size

        mesh= pf.load_mesh(mesh_path)

        #Initialize empty lists to store results for each time step
        div_list = []
        curl_list = []

        for dd in range(n):  # Looping over 0 to 30 for the time dimension
            v = vnod[dd, :].compute()
            u = unod[dd, :].compute()
    
            ##Calculating Curl##
            dv_dx = (ddx * v[elem]).sum(dim='n3') 
            du_dy = (ddy * u[elem]).sum(dim='n3')
            curl = dv_dx - du_dy

            ## Curl at Node ##
            curl = pf.tonodes(curl, mesh)
            curl = xr.DataArray(curl, coords=v.coords, dims=v.dims)
    
            ## Calculating divergence ##
            dv_dy = (ddy * v[elem]).sum(dim='n3') 
            du_dx = (ddx * u[elem]).sum(dim='n3') 
            div = du_dx + dv_dy

            ## Divergence at Node ##
            div = pf.tonodes(div, mesh)
            ## Changing div to a Xarray object ##
            div = xr.DataArray(div, coords=v.coords, dims=v.dims)
    
           # Append div and curl to lists
            div_list.append(div)
            curl_list.append(curl)

            # Combine the lists into Xarray DataArrays along the time dimension
            divergence = xr.concat(div_list, dim='time')
            curl = xr.concat(curl_list, dim='time')

            # Add variable attributes
            divergence.attrs['long_name'] = 'Divergence of Ocean surface water velocity'
            divergence.attrs['units'] = '1/s'

            # Create a Dataset containing the DataArray
            dv = xr.Dataset({'divergence': divergence})

            dv.to_netcdf(f"/work/bm1344/a270228/div_curl/fesom_divergence_{year}{mon}_daily-NG5.nc")

            # Add variable attributes
            curl.attrs['long_name'] = 'Curl of Ocean surface water velocity'
            curl.attrs['units'] = '1/s'

            # Create a Dataset containing the DataArray
            cl = xr.Dataset({'curl': curl})

            cl.to_netcdf(f"/work/bm1344/a270228/div_curl/fesom_curl_{year}{mon}_daily-NG5.nc")



/work/ab0995/a270088/meshes/NG5/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /work/ab0995/a270088/meshes/NG5/pickle_mesh_py3_fesom2
/work/ab0995/a270088/meshes/NG5/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /work/ab0995/a270088/meshes/NG5/pickle_mesh_py3_fesom2
/work/ab0995/a270088/meshes/NG5/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /work/ab0995/a270088/meshes/NG5/pickle_mesh_py3_fesom2
/work/ab0995/a270088/meshes/NG5/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /work/ab0995/a270088/meshes/NG5/pickle_mesh_py3_fesom2
/work/ab0995/a270088/meshes/NG5/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /work/ab0995/a270088/meshes/NG5/pickle_mesh_py3_fesom2
/work/ab0995/a2